In [19]:
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import re
from BetaPose import session_prep


######### Load session
class ACGUI_KIT:
  def __init__(self, url, size=(2400,1000)): 
    self.JOBID = ""; 
    self.url = url; 
    self.cofactors = ["SO4", "ACT", "EDO"]; 
    self.driver = webdriver.Firefox(); 
    self.driver.get(self.url); 
    self.driver.set_window_size(*size); 

    assert "ACGUI" in self.driver.title, "Is this an ACGUI url? "; 
    if "session_id" in url: 
      elem = self.driver.find_element(By.ID, "Session_loader"); 
      elem.click(); 
      tmp = [i for i in self.url.split('?') if "session_id" in i]
      self.JOBID = tmp[0].split("=")[1]; 
      time.sleep(1)
      
    else: 
      print("No job id found, you might be initiating a new session")
  def disableTooltips(self):
    for elm in self.driver.find_elements(By.ID, "hview"): 
      if 'Preferences' in elm.get_attribute('value').split():
        elm.click(); 
        btns = self.driver.find_elements(By.CLASS_NAME, "ui-button"); 
        for btn in btns: 
          if btn.text == "Pop-ups":
            btn.click(); 
            time.sleep(0.1)
            self.driver.find_element(By.ID, "TTPRadio2").click(); 
            for i in self.driver.find_elements(By.CLASS_NAME, "ui-button"): 
              if i.text == "Got it!": 
                i.click();
                break
            break
        
  def __check(self):
    assert len(self.JOBID) == 8, "Not a valid JOBID";
    
  def __clickSimulateDropdown(self):
    self.__check(); 
    simbutton = self.driver.find_element(By.ID, "simulatemenubutton"); 
    simbutton.click(); 
    time.sleep(0.1);
    
  def __clickDockingDropdown(self):
    self.__check(); 
    simbutton = self.driver.find_element(By.ID, "dockmenubutton"); 
    simbutton.click(); 
    time.sleep(0.1);
    
  def __ClickResidue(self, value): 
    resbuttons = self.driver.find_elements(By.CLASS_NAME, "resbutton"); 
    for res in resbuttons:
      if res.get_property("value") == value: 
        res.click(); 
        break
      if res.get_property("value") == resbuttons[-1].get_property("value"):
        print(f"{self.JOBID}: Warning, not found matched residue; "); 
  
  def __ClickUIButton(self, text):
    buttons = self.driver.find_elements(By.CLASS_NAME, "ui-button"); 
    for idx, btn in enumerate(buttons): 
      if btn.text == "Close":
        btn.click(); 
        break
      if idx == len(buttons)-1:
        print("Not found close button"); 
  
  def SelectResidue(self, resids):
    if isinstance(resids, str):
      self.ClickByID("showseq"); 
      self.__ClickResidue(resids);
      self.ClickByID("togglemenubutton"); 
      self.ClickByID("togglesel"); 
    elif isinstance(resids, list):
      for res in resids:
        print(f"Selecting {res}")
        self.ClickByID("showseq"); 
        self.__ClickResidue(res);
        self.__ClickUIButton("Close"); 
        self.ClickByID("togglemenubutton"); 
        self.ClickByID("togglesel"); 
    else: 
      print("Please provide a valid resid argument"); 
      
        
  def Fetch(self, pdbcode, lig=None): 
    pdbenter = self.driver.find_element(By.ID, "pdbenter")
    pdbenter.send_keys(pdbcode); 
    time.sleep(2)
    if lig != None: 
      if isinstance(lig, str) and lig == "auto":
        lig_menu = self.driver.find_element(By.ID, "pdbligand-button"); 
        lig_menu.click(); 
        lig_lst = self.driver.find_element(By.ID, "pdbligand-menu").find_elements(By.TAG_NAME, "li"); 
        print(f"{self.JOBID}: All available ligands: ", [i.text for i in lig_lst])
        for item in lig_lst: 
          if item.text != "none": 
            if item.text.split("_")[0] in self.cofactors:
              continue
            time.sleep(1); 
            ActionChains(self.driver).move_to_element(item).click().perform(); 
            break; 
        wat_range = self.driver.find_element(By.ID, "waterradius"); 
        wat_range.clear();
        wat_range.send_keys("0");
        time.sleep(1); 
    pdbsubmit = self.driver.find_element(By.ID, "pdbsubmit")
    pdbsubmit.click(); 
    time.sleep(4); 
    
  def OpenPrepareForm(self, settings={}): 
    simbutton = self.driver.find_element(By.ID, "simulatemenubutton"); 
    simbutton.click(); 
    time.sleep(0.1); 
    perpbutton = self.driver.find_element(By.ID, "simulateprep"); 
    perpbutton.click(); 
    if len(settings.keys()) > 0: 
      for key, val in settings.items():
        simparm = self.driver.find_element(By.ID, key); 
        simparm.clear(); 
        simparm.send_keys(val); 
        time.sleep(0.1)
        
  def SubmitPrepareForm(self): 
    self.__check(); 
    submit_btn = self.driver.find_elements(By.CLASS_NAME, "ui-button"); 
    
    for idx, btn in enumerate(submit_btn): 
      if btn.text == "Submit CAMPARI job":
        print(f"{self.JOBID}: submiting the preparation job"); 
        btn.click(); 
        break
      else: 
        continue
      if idx == len(submit_btn)-1:
        print("Not found CAMPARI job submission button")

  def UploadMol2(self, mol2token): 
    self.__check(); 
    if os.path.isfile(mol2token):
      session_prep.SubmitMOL2(mol2token, self.JOBID, mode="file")
    elif "@<TRIPOS>MOLECULE" in mol2token: 
      session_prep.SubmitMOL2(mol2token, self.JOBID, mode="str")
    else: 
      print("Fatal: Not found a valid mol2 file nor a valid mol2 string;")
      
  def ClickByID(self, ID, latent=1):
    item = self.driver.find_element(By.ID, ID); 
    try:
      item.click(); 
    except: 
      ActionChains(self.driver).move_to_element(item).click().perform(); 
    time.sleep(latent); 
    
  def OpenMDForm(self):
    st = time.perf_counter(); 
    self.__clickSimulateDropdown(); 
    # The structure should be loaded within 60 seconds according to rough estimation
    for i in range(600):   
      self.__clickSimulateDropdown(); 
      MDformbutton = self.driver.find_element(By.ID, "simulatemd"); 
      print(MDformbutton.get_attribute('class').split()); 
      if 'loadfilebutton' in MDformbutton.get_attribute('class').split():
        print(f"Element is enabled ({(i+1)*0.1} seconds) ({time.perf_counter() - st} seconds)")
        MDformbutton.click(); 
        time.sleep(0.1); 
        break
      else: 
        time.sleep(0.1); 
    
  def SetMDForm(self, settings={}):
    if len(settings.keys()) > 0: 
      for key, val in settings.items():
        if key in [
          'simTimestep', 'simNrSteps', 'simTrajOutInterval', 'sim_prodensemble', 
          'sim_prodengine', 'sim_prodpress', 'sim_prodtemp', 'simNrCopy', 'simOutFormat', 'simOutputGroup',
          "sim_batchselection"
                  ]:
          if not self.driver.find_element(By.ID, "SMDSet4div").is_displayed():
            self.ClickByID("SMDSet4"); 
            time.sleep(0.05)
        elif key == "equilnr":
          if not self.driver.find_element(By.ID, "SMDSet3div").is_displayed():
            self.ClickByID("SMDSet3"); 
            time.sleep(0.05)
          self.ClickByID("sim_removeequil"); 
          for i in range(int(val)):
            self.ClickByID("sim_addequil", latent=0.1); 
          continue
        elif re.match(r"equil[0-9]_", key) or key in [""]:
          if not self.driver.find_element(By.ID, "SMDSet3div").is_displayed():
            self.ClickByID("SMDSet3"); 
            time.sleep(0.05)
        
        item = self.driver.find_element(By.ID, key); 
        if item.tag_name == "select":
          options = item.find_elements(By.TAG_NAME, "option")
          for i in options: 
            if i.get_property("value") == val:
              item.click();
              time.sleep(0.05); 
              i.click();
              break; 
            if i.get_property("value") == options[-1].get_property("value"):
              print(f"Not found a valid choice for {key} valued {val}")

        elif item.tag_name == "input" and item.find_elements(By.XPATH, ".[@type='checkbox']"):
          if int(val) > 0: 
            item.click(); 
        elif item.tag_name == "input" and not item.find_elements(By.XPATH, ".[@type='checkbox']"):
          item.clear(); 
          item.send_keys(val); 
        
  def SubmitMDForm(self):
    self.__check(); 
    submit_btn = self.driver.find_elements(By.CLASS_NAME, "ui-button"); 
    
    for idx, btn in enumerate(submit_btn): 
      if btn.text == "Submit simulation job":
        print(f"{self.JOBID}: submiting the MD simulation job"); 
        btn.click(); 
        break
      else: 
        continue
      if idx == len(submit_btn)-1:
        print("Not found <Submit simulation job> submission button")
      
  def OpenSeedForm(self, refset, settings={}):
    self.SelectResidue(refset); 
    self.ClickByID("dockmenubutton"); 
    time.sleep(0.1); 
    self.ClickByID("dockseed"); 
    time.sleep(0.1); 
    if len(settings.keys()) > 0: 
      for key, val in settings.items():
        parm_input = self.driver.find_element(By.ID, key); 
        parm_input.clear(); 
        parm_input.send_keys(val); 
        time.sleep(0.1)
  def OpenCTDockForm(self, settings={}): 
    self.ClickByID("dockmenubutton"); 
    time.sleep(0.1); 
    self.ClickByID("dockctdock"); 
    time.sleep(0.1); 
    self.driver.switch_to.alert.accept();
    
    
  def close(self):
    self.driver.close();
    self.driver.quit();

In [20]:
PDBBind_path = "/home/yzhang/Downloads/refined-set"
PDBBind_path = "/home/miemie/Dropbox/PhD/project_MD_ML/PDBbind_v2020_refined"

pdbcode = "1Q65"; 
sess_id = "CsDJQucZ"; 
# acgui_url = f"http://130.60.168.149/ACyang/search2.html?session_id={sess_id}"
acgui_url = f"http://130.60.168.149/ACyang/search2.html?session_id={sess_id}"
# mol2file = f"{PDBBind_path}/{pdbcode.lower}/{pdbcode}_ligand.mol2"; 

testcase = ACGUI_KIT(acgui_url, size=(1500,1000)); 
#################Submit Preparation
# parms = {
#   "MAXLOOPL" : 0, 
#   "NRSTEPS" : 500, 
#   "MINI_MODE" : 1,
#   "PDB_TOLERANCE_A" : 50, 
#   "PDB_TOLERANCE_B" : "0.5 1.5"
# }
# testcase.Fetch(pdbcode, lig="auto"); 
# testcase.OpenPrepareForm(settings=parms); 
# testcase.SubmitPrepareForm();  

# testcase.UploadMol2(mol2file); 

################Submit MD
simsettings = {
  "sim_batchselection": 1, 
  "simTimestep": 2, 
  "simNrSteps" : 100000, 
  "simTrajOutInterval": 25000, 
  "sim_prodensemble": "npt", 
  "sim_prodengine": "gmxmd", 
  "sim_prodpress": 1, 
  "sim_prodtemp" : 300, 
  "simNrCopy": 1, 
  
  "simOutFormat": "netcdf", 
  "simOutputGroup": "all", 
  
  "equilnr":2,
  "equil0_nrsteps": 4000, 
  'equil1_ensemble': 'nvt',
  'equil1_temp': 300,
  'equil1_timestep': 2,
  'equil1_nrsteps': 10000,
  'equil1_tctime': 0.1,
  'equil2_ensemble': 'npt',
  'equil2_temp': 300,
  'equil2_timestep': 2,
  'equil2_nrsteps': 10000,
  'equil2_tctime': 0.1,
}

print(list(simsettings.keys()))

testcase.disableTooltips()
testcase.OpenMDForm()
testcase.SetMDForm(simsettings)
testcase.SubmitMDForm()
testcase.close()

#################Submit Docking
# POCKET = ["VAL 74", "THR 91"]; 
# seed_parms = {
#   "OPMODE" : "d",           # Docking mode 
#   "VECCHOICE" : "b",        # Vector type 
#   "ENTOTFILTER" : "-14",    # Energy filter 
#   "PROTDIEL" : "2",         # Dielectric constant 
#   "NRMOLPO" : "1",          # Top N poses per mol 
#   "NRCLUPO" : "0",          # Top N poses per cluster 
#   "SEEDSD_CHOICE" : "n",    # Whether or not to play steepest descent; 
#   "SEEDVF_MODE" : 1,        # Post run filtering; OFF 0, OR 1, AND 2; 
#   "SEEDVF_TOT" : "0.1",
#   "SEEDVF_ELE" : "0.1",
#   "SEEDVF_VDW" : "0.1",
#   "SEEDVF_TOTE" : "0.1",
#   "SEEDVF_ELEE" : "0.1",
#   "SEEDVF_VDWE" : "0.1",
#   "SEEDVF_RDES" : "0.1",
#   "SEEDVF_FDES" : "0.1",
#   "PSQLPDBCODE" : "5UUK",   # PDB code; 
#   "CONFO_RMS1" : 0.5,       # RMSD-like similarity shreshold; 
#   "CONFO_RMS2" : 0.05,      # Non-RMSD-like similarity shreshold; 
#   "CONFO_ATTS" : 10,        # Embeding attempts; 
#   "LOOKUP_DB" : "zinc_2020",   # Source database; 
# }

# testcase.OpenSeedForm(POCKET, settings=seed_parms); 

# #################Submit CT Docking
# ct_parms = {
#   "TNRSTEPS" : 2000,
#   "TMOL2RANDOMIZE" : "0",        # Disable randomization
  
#   "TTOTENFILTER" : "", 
#   "CTVF_MODE" : "", 
#   "CTVF_TOT" : "", 
#   "CTVF_ELEC" : "", 
#   "CTVF_VDW" : "", 
#   "CTVF_STRAIN" : "", 
#   "CTVF_BIAS" : "", 
#   "TSC_IMPSOLV" : "", 
#   "TSC_POLAR" : "", 
#   "" : "", 
#   "" : "", 
#   "" : "", 
#   "" : "", 
#   "" : "", 
#   "" : "", 
#   "TFREESTYLE" : "FMCSC_DYNAMICS 5", 
  
  
  
# }

# print(dir(testcase.driver))
# testcase.OpenCTDockForm(settings=ct_parms)



['sim_batchselection', 'simTimestep', 'simNrSteps', 'simTrajOutInterval', 'sim_prodensemble', 'sim_prodengine', 'sim_prodpress', 'sim_prodtemp', 'simNrCopy', 'simOutFormat', 'simOutputGroup', 'equilnr', 'equil0_nrsteps', 'equil1_ensemble', 'equil1_temp', 'equil1_timestep', 'equil1_nrsteps', 'equil1_tctime', 'equil2_ensemble', 'equil2_temp', 'equil2_timestep', 'equil2_nrsteps', 'equil2_tctime']
['loadfilebutton']
Element is enabled (0.1 seconds) (0.6553470338694751 seconds)
CsDJQucZ: submiting the MD simulation job


In [6]:
sess_id = "CsDJQucZ"; 
acgui_url = f"http://130.60.168.149/AC/search2.html?session_id={sess_id}"; 
testcase = ACGUI_KIT(acgui_url, size=(1500,1000)); 

#################Submit Docking
POCKET = ["VAL 74", "THR 91"]; 
seed_parms = {
  "OPMODE" : "e",           # Evaluation mode 
  "VECCHOICE" : "b",        # Vector type 
  "ENTOTFILTER" : "-18",    # Energy filter 
  "PROTDIEL" : "2",         # Dielectric constant 
  "NRMOLPO" : "1",          # Top N poses per mol 
  "NRCLUPO" : "0",          # Top N poses per cluster 
  "SEEDSD_CHOICE" : "y",    # Whether or not to play steepest descent; 
  "SEEDSD_MAXITER" : 1000,  # Steepest gradient descent iteration
  "SEEDSD_EPS" : 0.02,      # Stopping threshold 
  "SEEDVF_MODE" : 0,        # Post run filtering; OFF 0, OR 1, AND 2; 
  "SEEDVF_TOT"  : "1",
  "SEEDVF_ELE"  : "1",
  "SEEDVF_VDW"  : "1",
  "SEEDVF_TOTE" : "1",
  "SEEDVF_ELEE" : "1",
  "SEEDVF_VDWE" : "1",
  "SEEDVF_RDES" : "1",
  "SEEDVF_FDES" : "1",
  "PSQLPDBCODE" : "5UUK",        # PDB code; 
  "CONFO_NR"    : 0,             # Distable the conformer generation; 
  "CONFO_RMS1"  : 0.5,           # RMSD-like similarity shreshold; 
  "CONFO_RMS2"  : 0.05,          # Non-RMSD-like similarity shreshold; 
  "CONFO_ATTS"  : 10,            # Embeding attempts; 
  "LOOKUP_DB"   : "zinc_2020",   # Source database; 
}

testcase.OpenSeedForm(POCKET, settings=seed_parms); 

Selecting VAL 74
Selecting THR 91


In [ ]:
sess = ['1SWR','1Q65','2D3Z:A','2FLR','3D0B','2QE4',
        '2J7E:A','1K21','1K22','1P57','1E2K','3F8C',
        '3BGZ','1H1S','1ZGI','1M48','1N46','1MU6',
        '2ZB1','1O5E','1LKE','2V95','2J7G','1E2L',
        '1TA6']
# sess = [i.lower() for i in sess]
print(sess)

solved = ["1SWR", "2QE4", "2FLR", "3D0B", "2ZB1",
          "1M48", "3BGZ", "1E2K", "3F8C", "2V95",
          "1E2L", "2D3Z",  
         ]
Failed = ["C4001P57", "C4001K21", "C4001Q65", "C4001K22", "C4001O5E", 
          "C4001TA6", 
         ]

for pdb in sess: 
  if "1O5E" not in pdb:
    continue
  pdbcode = pdb; # .lower(); 
  PDBCODE = pdbcode.split(":")[0].upper()
  if PDBCODE in solved: 
    continue
  sess_id = f"C400{PDBCODE}"; 
  acgui_url = f"http://130.60.168.149/ACyang/search2.html?session_id={sess_id}"
  mol2file = f"{PDBBind_path}/{pdbcode}/{pdbcode}_ligand.mol2"; 

  testcase = ACGUI_KIT(acgui_url); 
  try: 
    testcase.Fetch(pdbcode, lig="auto"); 
    testcase.OpenPrepareForm(settings=parms); 
    testcase.SubmitPrepareForm();  
  except: 
    print(f"Skipping the operation on {sess_id}")


In [239]:
atoms[traj.top.select("@?H=")]
try:
  print(residues[1].__getattribute__("chainid"))
except: 
  print("no")
try: 
  print(atoms.__getattribute__("chainid"))
except:
  print("no")

no
no


In [22]:
import pandas as pd 

simsettings = {
  "sim_batchselection": 0, 
  
  "simTimestep": 2, 
  "simNrSteps" : 25000, 
  "simTrajOutInterval":25000, 
  "sim_prodensemble": "npt",
  "sim_prodengine": "gmxmd", 
  "sim_prodpress": 1,
  "sim_prodtemp" : 300,
  "simNrCopy": 1, 
  
  "simOutFormat": "netcdf", 
  "simOutputGroup": "all", 
  
  "equilnr":5,
  "equil0_nrsteps": 5000, 
  'equil1_ensemble': 'nvt',
  'equil1_temp': 370,
  'equil1_timestep': 0.1,
  'equil1_nrsteps': 8000,
  'equil1_tctime': 0.0005,
  'equil2_ensemble': 'nvt',
  'equil2_temp': 345,
  'equil2_timestep': 0.5,
  'equil2_nrsteps': 8000,
  'equil2_tctime': 0.0025,
  'equil3_ensemble': 'nvt',
  'equil3_temp': 300,
  'equil3_timestep': 0.75,
  'equil3_nrsteps': 4000,
  'equil3_tctime': 0.01,
  'equil4_ensemble': 'npt',
  'equil4_temp': 298,
  'equil4_timestep': 1,
  'equil4_nrsteps': 3000,
  'equil4_tctime': 0.05,
  'equil5_ensemble': 'npt',
  'equil5_temp': 298,
  'equil5_timestep': 2,
  'equil5_nrsteps': 2000,
  'equil5_tctime': 0.1,
}
  
def domd_selenium(jobid):
  time.sleep(int(time.perf_counter().__str__()[-3:])/3000);
  acgui_url = f"http://130.60.168.149/ACyang/search2.html?session_id={jobid}"; 
  testcase = ACGUI_KIT(acgui_url, size=(1600, 900)); 
  testcase.disableTooltips()
  testcase.OpenMDForm(); 
  testcase.SetMDForm(simsettings); 
  testcase.SubmitMDForm(); 
  testcase.close(); 
  return

  

108 sessions will be submitted to the server:  C4001MQ5 C4002IDW C4001SWG C4001GPN C4001WS4 C4001LPZ C4002A8G C4001Y3N C4007STD C4001FKI C4001XK9 C4001UWF C4001QBO C4002JDS C4002CSN C4002D0K C4001B1H C4001OCQ C4003B68 C4001B05 C4001O5A C4003BGQ C4001ELD C4003B67 C4001Y6Q C4001HMS C4002GV6 C4001DHI C4001ERB C4002F80 C4001FH9 C4001FCY C4001ELA C4001F4E C4001JET C4002DRC C4001HMR C4003GST C4001LBK C4001PZI C4003BGB C4005ER1 C4003BEX C4001OGZ C4001OWH C4001UGX C4002OXX C4003DJK C4001V2R C4001G74 C4002F35 C4002OGY C4002V00 C4001KDK C4001C5X C4001W3K C4001N4H C4002E2R C4001LF2 C4005STD C4001V0K C4001FH7 C4001V0L C4002J34 C4001MFI C4002GST C4002FQW C4002F34 C4002QM9 C4002CGF C4002HB1 C4002IWX C4001Q8T C4003EKO C4001ADL C4001D6V C4001FCZ C4001FL3 C4002QBQ C4002FQX C4001XHY C4001JQY C4001F4X C4001NO6 C4001ENU C4002GSS C4002QBU C4001TD7 C4001OYQ C4001P1O C4001K4H C4001EJN C4002STD C4001OD8 C4001ZOG C4001LEE C4003B66 C4002F1G C4002EXM C4001VYF C4002E7F C4001BGQ C4001H22 C4002BOH C4003D7Z C4002PSV

# Single MD submission

In [ ]:
# loadfilebutton
domd_selenium("C4001E2L")


Preferences found
['loadfilebutton']
Element is enabled (0.1 seconds)
C4001E2L: submiting the MD simulation job


# Batch submission

In [ ]:
# table = pd.read_csv("/home/yzhang/Documents/Personal_documents/BetaPose/AfterPrep_108_142.csv"); 
# session_ids = [row.sess_id for idx, row in table.iterrows() if row.TMScore > 0]
# print(f"{len(session_ids)} sessions will be submitted to the server: ", *session_ids); 

sids = ['C4001MQ5', 'C4002IDW', 'C4001WS4', 'C4001LPZ', 'C4002A8G', 'C4001Y3N', 'C4007STD', 'C4001XK9', 'C4001UWF', 'C4001QBO', 'C4002JDS', 'C4002CSN', 'C4001B1H', 'C4001OCQ', 'C4003B68', 'C4001B05', 'C4001O5A', 'C4003BGQ', 'C4001ELD', 'C4003B67', 'C4002QE4', 'C4001Y6Q', 'C4001HMS', 'C4002GV6', 'C4001FH9', 'C4001FCY', 'C4001ELA', 'C4001F4E', 'C4001JET', 'C4002DRC', 'C4001HMR', 'C4003GST', 'C4001PZI', 'C4001E2K', 'C4003BGB', 'C4005ER1', 'C4003BEX', 'C4001OGZ', 'C4001UGX', 'C4002OXX', 'C4003DJK', 'C4001V2R', 'C4001G74', 'C4002F35', 'C4002OGY', 'C4001KDK', 'C4001C5X', 'C4001W3K', 'C4001N4H', 'C4002E2R', 'C4001LF2', 'C4005STD', 'C4001V0K', 'C4001FH7', 'C4001V0L', 'C4001MFI', 'C4002GST', 'C4002FQW', 'C4002F34', 'C4002QM9', 'C4002CGF', 'C4002HB1', 'C4002IWX', 'C4001Q8T', 'C4002ZB1', 'C4003EKO', 'C4001ADL', 'C4001D6V', 'C4001FCZ', 'C4001FL3', 'C4002QBQ', 'C4002FQX', 'C4001XHY', 'C4001JQY', 'C4001F4X', 'C4001NO6', 'C4001ENU', 'C4002GSS', 'C4002QBU', 'C4001TD7', 'C4001OYQ', 'C4001K4H', 'C4001EJN', 'C4002STD', 'C4001OD8', 'C4001ZOG', 'C4002V95', 'C4003B66', 'C4002F1G', 'C4002EXM', 'C4001VYF', 'C4001E2L', 'C4002E7F', 'C4001BGQ', 'C4002BOH', 'C4003D7Z', 'C4002PSV', 'C4001ELE', 'C4001HMT']
for sess in sids[77:]: 
  print(f"Running simulation on the session: {sess} ({sids.index(sess)})"); 
  domd_selenium(sess); 
  
  

In [101]:
a="ipynb"
!ls *$a

Evaluations.ipynb    Pairwise_distance.ipynb	Test_clustering.ipynb
Label_DataSet.ipynb  PDBBind_selection.ipynb	Untitled.ipynb
Language.ipynb	     Selenium_Submission.ipynb


In [59]:
f = "/home/miemie/Downloads/TSpe4aeM_lig.mol2"

file1 = open(f, 'r'); a = file1.read(); file1.close(); 
atoms = [float(i.strip().split()[-1]) for i in a.split("@<TRIPOS>ATOM")[1].split("@<TRIPOS>BOND")[0].strip("\n").split("\n")]; 
print("\n".join([str(i) for i in atoms]))



0.2249
0.0167
0.0047
-0.0008
-0.1579
0.0894
-0.5442
-0.5442
0.0226
0.0986
-0.3471
0.1121
-0.3865
0.1285
-0.3847
0.1666
-0.2855
0.1004
-0.2306
0.1473
0.1472
-0.3413
-0.2188
0.1205
-0.2159
0.1674
0.2001
0.2001
0.2001
0.0915
0.0332
0.0332
0.0378
0.0378
0.0402
0.0402
0.0632
0.0647
0.21
0.0665
0.2101
0.0866
0.103
0.1438
0.1438
0.1048
